In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Does nn.Conv2d init work well

In [2]:
from exp.nb_02 import *

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()

In [5]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape, x_valid.shape

(torch.Size([60000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [7]:
n, *_ = x_train.shape
c = y_train.max() + 1
nh = 32
n, c

(60000, tensor(10))

In [8]:
l1 = nn.Conv2d(1, nh, 5)

In [9]:
x = x_valid[:100]
x.shape

torch.Size([100, 1, 28, 28])

In [10]:
def stats(x): return x.mean(), x.std()

In [12]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [13]:
l1.bias.shape

torch.Size([32])

In [11]:
stats(l1.weight), stats(l1.bias)

((tensor(-0.0025, grad_fn=<MeanBackward0>),
  tensor(0.1158, grad_fn=<StdBackward0>)),
 (tensor(-0.0145, grad_fn=<MeanBackward0>),
  tensor(0.1054, grad_fn=<StdBackward0>)))

In [14]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self) -> None
Docstring: <no docstring>
Source:   
    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/envs/fastbook/lib/python3.8/site-packages/torch/nn/modules/conv.py
Type:      function


In [15]:
torch.nn.init.kaiming_uniform_??

Signature:
torch.nn.init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of th

In [16]:
t = l1(x)
stats(t)

(tensor(-0.0218, grad_fn=<MeanBackward0>),
 tensor(0.6243, grad_fn=<StdBackward0>))

In [17]:
torch.nn.init.kaiming_normal_??

Signature:
torch.nn.init.kaiming_normal_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \frac{\text{gain}}{\sqrt{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
         

In [21]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0448, grad_fn=<MeanBackward0>),
 tensor(1.0420, grad_fn=<StdBackward0>))

In [22]:
import torch.nn.functional as F

In [23]:
def f1(x, a=0): return F.leaky_relu(l1(x), a) # l1 is Conv2D

In [24]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5347, grad_fn=<MeanBackward0>),
 tensor(0.9670, grad_fn=<StdBackward0>))

In [25]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2112, grad_fn=<MeanBackward0>),
 tensor(0.3722, grad_fn=<StdBackward0>))

In [26]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [27]:
rec_fs = l1.weight[0][0].numel()
rec_fs

25

In [28]:
nf, ni, *_ = l1.weight.shape
nf, ni

(32, 1)

In [29]:
fan_in = ni * rec_fs
fan_out = nf  *rec_fs
fan_in, fan_out

(25, 800)

In [32]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [37]:
math.sqrt(5.)

2.23606797749979

In [33]:
gain(1), gain(0), gain(0.1), gain(0.01), gain(math.sqrt(5.)) # gain(1) linear, gain(0) Relu

(1.0,
 1.4142135623730951,
 1.4071950894605838,
 1.4141428569978354,
 0.5773502691896257)

In [55]:
init.uniform_(torch.zeros(10000), -1., 1.).std()

tensor(0.5790)

In [56]:
init.normal_(torch.zeros(10000)).std()

tensor(0.9961)

In [57]:
1 / math.sqrt(3.)

0.5773502691896258

In [65]:
def kaiming2(x, a, use_fan_out=False):
    nf, ni, *_ = x.shape
    rec_fs = x[0][0].shape.numel()
    fan = nf * rec_fs if use_fan_out else ni * rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound, bound)

In [66]:
kaiming2(l1.weight, a=0)
stats(f1(x))

(tensor(0.5328, grad_fn=<MeanBackward0>),
 tensor(1.0040, grad_fn=<StdBackward0>))

In [67]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.1966, grad_fn=<MeanBackward0>),
 tensor(0.3780, grad_fn=<StdBackward0>))

In [68]:
class Flatten(nn.Module):
    def forward(self, x): return x.view(-1)

In [69]:
m = nn.Sequential(nn.Conv2d(1, 8, 5, stride=2, padding=2), nn.ReLU(),
                 nn.Conv2d(8, 16, 3, stride=2, padding=1), nn.ReLU(),
                 nn.Conv2d(16, 32, 3, stride=2, padding=1), nn.ReLU(),
                 nn.Conv2d(32, 1, 3, stride=2, padding=1), nn.ReLU(),
                 nn.AdaptiveAvgPool2d(1),
                 Flatten()
                 )

In [90]:
y = y_valid[:100].float()

In [91]:
t = m(x)
stats(t)

(tensor(0.5235, grad_fn=<MeanBackward0>),
 tensor(0.2336, grad_fn=<StdBackward0>))

In [92]:
loss = mse(t,y)
loss.backward()

In [93]:
stats(m[4].weight.grad)

(tensor(-0.1038), tensor(0.5129))

In [94]:
for l in m:
    if isinstance(l, nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [95]:
t = m(x)
stats(t)

(tensor(0.3053, grad_fn=<MeanBackward0>),
 tensor(0.1758, grad_fn=<StdBackward0>))

In [96]:
loss = mse(t,y)
loss.backward()
stats(m[0].weight.grad)

(tensor(-0.6574), tensor(0.6340))

In [97]:
stats(m[4].weight.grad)

(tensor(-0.1090), tensor(0.5487))